In [26]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image
from src.train_pm import Dataset, double_conv, LeUNet, StandardNet, EnsembleNet, EPAPLN, ResNetUNet, EnsembleLeUNet
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from src.preprocessing.trans_func import *
import torchvision.models as models
import Augmentor
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
p = Augmentor.Pipeline("/scratch/ab9738/pollution_img/new_images/")

Initialised with 511 image(s) found.
Output directory set to /scratch/ab9738/pollution_img/new_images/output.

In [28]:
p.crop_random(probability=1.0,percentage_area=0.5)

In [29]:
for i in range(10):
    p.sample(511)

Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E872C2340>: 100%|██████████| 511/511 [00:14<00:00, 35.74 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E8787FD60>: 100%|██████████| 511/511 [00:14<00:00, 35.83 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E862238B0>: 100%|██████████| 511/511 [00:14<00:00, 35.90 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147F6E96C730>: 100%|██████████| 511/511 [00:14<00:00, 35.98 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E1759A1F0>: 100%|██████████| 511/511 [00:14<00:00, 35.66 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E8787F100>: 100%|██████████| 511/511 [00:13<00:00, 36.57 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x1500 at 0x147E17580520>: 100%|██████████| 511/511 [00:14<00:00, 35.91 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=2000x15

In [49]:
aug_list = os.listdir('/scratch/ab9738/pollution_img/new_images/output')

In [50]:
aug_list

['new_images_original_20220121_170438___28.631901_77.34521___[org].jpg_48f4dd12-392e-41e9-953c-94fdc6f82ae9.jpg',
 'new_images_original_20220121_185143___28.584698_77.229692___[org].jpg_6f5a7464-b05c-4832-895f-fd8046c0dbb1.jpg',
 'new_images_original_20220121_182625___28.628236_77.271899___[org].jpg_b01f0dc0-64ac-4f66-9c1a-5bc81db70c57.jpg',
 'new_images_original_20220122_171251___28.628434_77.320904___[org].jpg_a06891f0-a33e-4938-95b8-1f3f22461e8e.jpg',
 'new_images_original_20220121_171653___28.648781_77.317658___[org].jpg_2af6bc50-8fdd-466c-8c7d-f0233c9a7d6a.jpg',
 'new_images_original_20220122_164422___28.660252_77.376252___[org].jpg_9ec85258-77a7-48c8-80fb-2ac6657f7646.jpg',
 'new_images_original_20220121_173552___28.629879_77.312289___[org].jpg_5f026a2a-27af-4e7f-8a76-db8dfbd4cbc3.jpg',
 'new_images_original_20220122_155653___28.636439_77.441936___[org].jpg_2363f035-c66a-4fd4-80b1-fc9594a2d79c.jpg',
 'new_images_original_20220122_174350___28.569327_77.328293___[org].jpg_a15fb0b5-

In [51]:
aug_original = aug_list.copy()

In [52]:
aug_list[0][20:].split('___')[2][:6]

'[org].'

In [53]:
for i, imgname in enumerate(aug_list):
    imgname = imgname[20:]
    t = imgname.split('___')
    t[2] = t[2][:5]+str(i)+'.jpg'
    imgname = '___'.join(t)
    aug_list[i] = imgname

In [56]:
aug_dir = '/scratch/ab9738/pollution_img/new_images/output'

In [58]:
os.path.join(aug_dir, aug_list[0])

'/scratch/ab9738/pollution_img/new_images/output/20220121_170438___28.631901_77.34521___[org]0.jpg'

In [54]:
aug_list

['20220121_170438___28.631901_77.34521___[org]0.jpg',
 '20220121_185143___28.584698_77.229692___[org]1.jpg',
 '20220121_182625___28.628236_77.271899___[org]2.jpg',
 '20220122_171251___28.628434_77.320904___[org]3.jpg',
 '20220121_171653___28.648781_77.317658___[org]4.jpg',
 '20220122_164422___28.660252_77.376252___[org]5.jpg',
 '20220121_173552___28.629879_77.312289___[org]6.jpg',
 '20220122_155653___28.636439_77.441936___[org]7.jpg',
 '20220122_174350___28.569327_77.328293___[org]8.jpg',
 '20220122_172802___28.585753_77.310184___[org]9.jpg',
 '20220121_181404___28.639837_77.293828___[org]10.jpg',
 '20220121_183555___28.621091_77.237763___[org]11.jpg',
 '20220122_161357___28.687843_77.450551___[org]12.jpg',
 '20220121_172200___28.658526_77.310567___[org]13.jpg',
 '20220121_184942___28.587667_77.233977___[org]14.jpg',
 '20220121_172725___28.64391_77.299134___[org]15.jpg',
 '20220122_175332___28.54459_77.327152___[org]16.jpg',
 '20220121_165302___28.630324_77.441666___[org]17.jpg',
 '202

In [55]:
import subprocess

In [59]:
for i in range(len(aug_list)):
    subprocess.call(['mv', os.path.join(aug_dir, aug_original[i]), os.path.join(aug_dir, aug_list[i])])